In [9]:
import pytesseract
from pdf2image import convert_from_path, convert_from_bytes
import cv2
import re
from datetime import datetime
import os
import itertools



class Viscardi:
    pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
    def __init__(self):
        self.config= '--psm 4  -c preserve_interword_spaces=1 tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.[]|,,.~â ÃÂç'
        self.tesseract_language = "por"
        self.lista_produtos = []
        self.imagem =  r'C:\\Users\\Guilherme\\Pictures\\readerpdf\\imagens\\'
        self.pdffile =  r'C:\\Users\\Guilherme\\Pictures\\readerpdf\\pdffiles\\VISCARDI (1).pdf'
        self.dataatual = str(datetime.today().strftime('%Y-%m-%d %H:%M'))
        self.idmarca = '21'
        self.listadicts = []

    def converter_imgpdf(self):

        images = convert_from_path(self.pdffile, 200, poppler_path=r'C:\\Users\\Guilherme\\Pictures\\readerpdf\\poppler-0.68.0\\bin')
        for i in range(len(images)):
            images[i].save(self.imagem+r'\viscardi'+ str(i) +'.jpg', 'JPEG')
    
    
    def search_files(self):
        imagem = [x for x in os.listdir(self.imagem) if 'viscardi']
        return imagem
    
    def get_referencias(self, sku):
        return str(sku['SKU'].strip())


    def reader_imagem(self):
        search_valores = re.compile(r'\s\d+\s\d+\,\d+')
        imagens = self.search_files()
        try:
            for imagem in imagens:
                file = cv2.imread(os.path.join(self.imagem,imagem))
                
                imagemgray = cv2.cvtColor(file, cv2.COLOR_BGR2RGB)

                texto = pytesseract.image_to_string(imagemgray, lang= self.tesseract_language,config=self.config)
            
                remove_esp = re.sub(r'\n',' ',texto)
                remove_esps = re.sub(r'\s+',' ',remove_esp)
            
                valores  = search_valores.findall(remove_esps)
                for valor in valores:
                    dict_itens = {}
                    valor = str(valor).strip().split(' ')
                    sku = 'Viscardi-'+str(valor[0])
                    saldo = float(str(valor[-1]).strip().replace(",","."))
                    dict_itens['SKU'] = sku
                    dict_itens['SALDO'] = saldo
                    dict_itens['DATA'] = self.dataatual
                    dict_itens['IDMARCA'] = self.idmarca

                    self.lista_produtos.append(dict_itens)
                    
        except:
            print("erro reader imagem")

        return self.lista_produtos
    
    def group_values(self):
        dicts = self.reader_imagem()
        lista_produtos_final = []
        agrupa_produtos = itertools.groupby(dicts, self.get_referencias)
        for sku, saldos in agrupa_produtos:

            lista = []

            for saldo in saldos:
                try:
                    sku = saldo['SKU']
                    saldo = float(str(saldo['SALDO']).strip().replace(".","").replace(",",".").replace("'",""))
                except:
                    sku = 'notfound'
                    saldo = float(0)
                lista.append(saldo)

            dict_items={

                    'SKU':sku,
                    'SALDO':sum(lista),
                    'IDMARCA':self.idmarca,
                    'DATA':self.dataatual
            }
            lista_produtos_final.append(dict_items)
        print(lista_produtos_final)


viscardi = Viscardi()
viscardi.converter_imgpdf()
viscardi.group_values()



[{'SKU': 'Viscardi-224', 'SALDO': 1900.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-239', 'SALDO': 880.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-85', 'SALDO': 530.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-110', 'SALDO': 1200.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-113', 'SALDO': 1980.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-123', 'SALDO': 1650.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-143', 'SALDO': 2130.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-21', 'SALDO': 3500.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-270', 'SALDO': 700.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-288', 'SALDO': 3470.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-289', 'SALDO': 8060.0, 'IDMARCA': '21', 'DATA': '2022-08-11 22:34'}, {'SKU': 'Viscardi-292', 'SALDO': 2000.0, 'IDMAR